In [3]:
import pandas as pd
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import OllamaLLM
from langchain_classic.chains import RetrievalQA
from langchain_classic.prompts import PromptTemplate

pd.set_option('display.max_colwidth', None)

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector_db = Chroma(
    persist_directory=r"D:\pY\InsuranceRAG\local_db",
    embedding_function=embeddings,
    collection_name="policy_master_collection"
)

llm = OllamaLLM(model="llama3:8b-instruct-q2_K",num_ctx=2048,  # Limits memory spike by restricting context window
    temperature=0.1)

template = """
You are a professional insurance auditor. Use the provided Policy context to answer the user's question. 
Stay strictly within the provided text. If you don't know, say it's not in the documents.

Context: {context}
Question: {question}
Answer:
"""

retriever = vector_db.as_retriever(search_kwargs={"filter": {"source_type": "Policy"}})
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": PromptTemplate.from_template(template)}
)

try:
    response = qa_chain.invoke("tell me where all policy can be covered, countries i mean?")
    print("🔎 AUDIT RESULT:")
    print(response["result"])
except Exception as e:
    print(f"❌ Execution Error: {e}")

🔎 AUDIT RESULT:
According to the Policy context, your cover does not apply to Iraq, Afghanistan, Somalia and in countries where war has been declared or after it has been recognised as a war zone by the United Nations.

So, based on this information, I would say that all policy coverage is excluded for those countries.


In [2]:
# 0. PRE-REQUISITE CHECK: Run this to ensure all libraries are installed
import importlib.util
import sys
import subprocess

required_libraries = [
    "langchain",            # Core orchestrator (RetrievalQA)
    "langchain_community",  # Community integrations
    "langchain_huggingface",# Embedding logic
    "langchain_chroma",     # Vector DB logic
    "langchain_ollama",     # Ollama connection
    "chromadb",             # Native DB client
    "pymupdf",              # PDF loading
    "pandas"                # Data display
]

def install_missing():
    missing = []
    for lib in required_libraries:
        # Check if the module is available
        spec = importlib.util.find_spec(lib.replace("-", "_"))
        if spec is None:
            missing.append(lib)
    
    if missing:
        print(f"📦 Missing libraries found: {missing}. Installing now...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", *missing])
        print("✅ Installation complete. Please RESTART YOUR KERNEL now!")
    else:
        print("✅ All pre-requisite libraries are already installed.")

install_missing()

✅ All pre-requisite libraries are already installed.


In [3]:
import sys
print ("🐍 Python Executable Path:")
print(sys.executable)

import langchain
print("📚 LangChain Version:")
print(langchain.__version__)

🐍 Python Executable Path:
d:\pY\InsuranceRAG\venv\Scripts\python.exe
📚 LangChain Version:
1.2.9
